# Tutorial 6: Basics of netCDF file manipulation

In previous tutorials, we've worked with discrete data such as hourly air quality measurements. In this tutorial, we'll move beyond discrete data into continuous or gridded data. This method can  be used with model data, reanalysis data, satellite data, or any dataset that is continuous or gridded over the area of interest.   

By the end of this tutorial, you will be able to:
* open netCDF files with Xarray
* distinguish between code for DataArrays and Datasets
* identify the properties of an Xarray DataArray and Dataset
* create a plot using continuous or gridded data
* manipulate data using array properties

### How to deal with 2D or 3D data in Python

So far, we've worked with lists and tables, which are examples of 1D (lists) and simple 2D (tables) data. But what if we have more complex 2D data, or even 3D data? pandas can no longer handle this type of information. Instead, we use other packages like *Xarray*.

In [ ]:
# for data manipulation and opening files
import xarray as xr
import pandas as pd

# for plotting
import matplotlib.pyplot as plt

<font color=red> **Note:**</font> If you get an error message saying that you do not have the necessary libraries, such as h5netcdf or pseudonetcdf, then you will need to install them. To do this, follow these instructions:
1. Open either a new terminal window (Mac) or Anaconda prompt (PC).
2. Copy and paste `conda install -c conda-forge netCDF4`.
3. Hit enter.
4. When prompted, hit `y`.
5. Once the installation is complete, repeat steps 2-4 for all necessary libraries. These may include h5netcdf, PyNIO, h5py, and PseudoNetCDF. If they are already installed, conda will tell you the package is up-to-date.

### Downloading data

Files of large gridded datasets can be too large to easily share. Instead, you will need to access the data yourself using the following steps. In later tutorials, we will show more ways to access data for multiple time periods in a loop. For now, we will individually access one file at a time. 

1. Go to https://psl.noaa.gov/data/gridded/data.narr.html. 
2. Check out the different data available. In the list of data, under "Options" you can click the graph icon to view a plot of the data in a new window. There are several reanalysis data products for meteorological data. For this tutorial, we will be using mean daily Planetary Boundary Layer (PBL) height. The code examples will be specific to this data, but you can choose any data product and just change the variable names.    
3. Once you have selected a data product from the left hand list, click on that data name. A drop-down list should appear of each URL extension, ending with the year and ".nc". Choose a time period of interest. This tutorial uses the 2020 data. 
    - **Reminder:** If you choose to work with a different data product or year, you will need to change the file link below and the variable names. 
4. Click on that URL extension of your year of interest. This will take you to the THREDDS Server page.
5. Click the OPeNDAP URL option (top link in the list). 
6. Copy the Data URL on the OPeNDAP page. 
7. Paste the URL below in the `file = 'https://` line. Change any other variables if you picked a different dataset. 

### Opening files with xarray

In [ ]:
file = 'http://psl.noaa.gov/thredds/dodsC/Datasets/NARR/Dailies/monolevel/hpbl.2020.nc'

In [ ]:
# use xarray to open the file you chose
data = xr.open_dataset(file,engine='netcdf4')

<font color=red> **Note:**</font> You might get a SerializationWarning when you run the previous line of code. This warning is simply saying that Xarray is reading all instances of $\pm$ 9.96921e+36 as Not a Number (NaN). We can check the validity of this on the NCEP info page https://psl.noaa.gov/data/gridded/data.narr.html. On the right hand side under Missing Data, missing values are indeed replaced with $\pm$ 9.96921e+36. 

Xarray can be described as a multi-dimensional version of pandas. Like pandas, the data can be indexed and sorted based on location or name. However, Xarray can do these things with data of three dimensions or more. Let's explore some of the ways Xarray organizes the data.

In [ ]:
# what does this data look like? This line of code may take a second to run - don't worry!
data

Ok, there's a lot going on here. Let's break these different things down a bit.    

**Array Type:** The very first thing that is shown in the above print out is 'xarray.Dataset'. There are two main types of arrays that xarray can handle: DataArrays and Datasets. A Dataset is a collection of DataArrays. Imagine that a DataArray is a rubix cube. It is a 3D (or 2D or 4D or however shaped) array of data. A Dataset would therefore be a collection of rubix cubes. Imagine you ordered a shipment of 30 rubix cubes. Each individual rubix cube is a DataArray, and the box that all the cubes were shipped in is the Dataset. We can select one DataArray using its variable name. There are some data selection methods that only work on DataArrays, which is why it is important to distinguish between DataArrays and Datasets.

**Dimensions:** The dimensions are the directions that the file has, or the names of the axes. There are four dimensions listed in this file: time, x, y, and nbnds. Since this is meteorological data, we can think of these as latitudes or rows and longitudes or columns. There are 277 y, which means there are 277 lines of latitude in the grid. There are 349 x, or 349 lines of longitude. The time dimension has 366 values. We selected daily means over a year, and 2020 is a leap year so 366 total days.  

**Coordinates:** Coordinates are labels for each step of a dimension. This particular file has three of the same coords as in dims: x, y, and time. However, nbnds is not in the coord list, and now there are lat and lon coords. Next to each coord is the number and name of the dimensions in that coord. Time, x, and y only have one dimension and those dimensions have the same name. Click on the piece of paper next to each of these coordinates. What does it say? In comparison, lat and lon have two dimensions: (y,x). This is because any point on a map will have a unique latitude and longitude combination. Each grid point in this Dataset will also have a unique latitude and longitude combination, which is made up of x and y values. 

**Data Variables:** These are the variables of interest. This is the actual data in your file. For example, there is "hpbl" that might be of interest to us. In some Datasets, the data variables have unusual names, which can make understanding the data difficult. Xarray is great because it allows you to have metadata for each data variable, which might help to explain the data a little more. Click on the picture of a piece of paper next to the "hpbl" variable name. What does it say? Some of the data variables have useful metadata under this piece of paper.    
Next, click on the picture of 3 stacked circles beside a data variable. What should appear is an array of the data. These are the actual values of this data variable.  

**Indexes:** Indexes are not always in an Xarray Dataset. Dimensions and coords are more useful in Xarray than the indexes, but you might find Datasets that include them. In this case, it is the three dimensions of the data (x,y, time) with a pandas Series of the values for each dimension that is listed in the coords. 

**Attributes:** Finally, Xarray has more metadata stored in the Attributes. Some of this information can be helpful, such as the lat and lon corners. Knowing the shape and map projection is important for properly formatting the data for visualization.

If you are interested in knowing just the attributes or just the coordinates or just the dims of an Xarray DataArray, you can call this information specifically.

In [ ]:
# here, we call the dimensions using the key word "dims"
data.dims

In [ ]:
# here, we call the coordinates using the key word "coords"
data.coords

In [ ]:
# here, we use the key word "attrs". Can you guess what this key word calls?
data.attrs

In [ ]:
# here, we call the data variables using the key word "var"
data.var

### Selecting and indexing data

Remember earlier when we said that DataArrays and Datasets are different? Indexing and selecting data is one of the places that distinguish DataArrays and Datasets. Right now, we have a Dataset. This is a collection of DataArrays. We can select one of these DataArrays from the whole Dataset using the variable name.

In [ ]:
# get a DataArray from our Dataset
# like using column names in pandas, we can select specific DataArrays using the array variable name
var = data['hpbl']

In [ ]:
# look at the data
var

**Knowledge Check:** How is this print out different from the print out of data? Why is it different? Take a moment here to really think through the differences between var and data and what they mean. Some information you might want to think about for the next few code cells:
1. What type array is this?
2. How many dimensions are there?

The biggest difference between var and data is that var is only one variable from the Dataset, but data has all the data variables. Therefore, var is a DataArray.

Like pandas, there are several ways to select data in Xarray. We can select the dimension of the data using positions or using names. Using positions, this is like saying "I want the data in axis 1". Using names, this is like saying "I want the data in the longitude". For either of these methods, you can use an integer or a label to select the index. We'll try a few examples below.

In [ ]:
# positional indexing of dimensions using integers
# translation: use a number to select the axis
var[0,:,:]

In the example above, we used square brackets ```[]``` to say "Take the dims of this DataArray." We then used colons ```:``` to say "Take all points in this dimension" and an integer ``0`` to say "take the 0th point in this dimension." The code ```var[0,:,:]``` therefore is saying "Take only the first time value, but all the values in the x and y dimensions." You can see how this was done by looking at the new dims of the data.

In [ ]:
# dimension name indexing using integers
var.isel(time=0)

In this example, we used the `isel` (integer select) command to select only the 0th time value of the DataArray. This is the same result as the first example.

In [ ]:
# another way to index using dimension names and integers
var[dict(time=0)]

You should now be able to see how Xarray, like pandas, has multiple methods to select and index data. This is only true for DataArrays. Datasets, because they are more complex, cannot be indexed using positions. Dimension names must be used. 

Next, let's slice our data.  
A slice is what it sounds like: it is a piece of the data that we are cutting out from the whole. `slice(None, 3)` says "Cut a piece of this data, starting at the 0th position (None) and ending at the 3rd position (3)." We used the dimension name "y" to tell xarray which dimension to index on. So `y=slice(None,3)` is saying "Cut a piece of the data along the y dimension, starting at the 0th position and ending at the 3rd position." This is how we cut out the first 3 y's of the data.   
In the cell below, we print out the dimensions of the original data and of some sliced data. Do the dimensions printed match your expectations?

In [ ]:
print(data.dims)
print(data[dict(y=slice(None, 30))].dims)
print(data[dict(x=slice(25,100))].dims)

**Knowledge Check:** If you see code that has square brackets `data[:]`, is that code selecting data using positional indexing or dimension name indexing? Which type of array can use this indexing method?

Like pandas, `[:]` indicates you are using positional indexing. The brackets say "of all the data, select based on the following positions", and the colon says "go from this first position to this last position when selecting the data".   
**In Xarray, only a DataArray can use positional indexing. For a Dataset, you must use dimension names.** 

### Data visualization

Now that we know how to select the data of interest, let's work on visualizing our data. Xarray has built-in plotting abilities, but these abilities are shortcuts of the same Matplotlib commands we learned in tutorials 3 and 4. To be more explicit in how we are visualizing and manipulating the data, we will continue to use the Matplotlib commands.

In [ ]:
# select a variable of interest for plotting
hpbl = data['hpbl']

In [ ]:
# look at the data
hpbl

This data is the height of the planetary boundary layer in meters from the surface. The dimensions of the data are x, y, and time. We could plot this data as rows (latitudes) and columns (longitudes) using a grid of 349 by 277 or we could make line plots of average heights over one of these dimensions. 

If you're having a hard time visualizing the data at this point, it might be helpful to go back to NCEP NARR and plot your data there https://psl.noaa.gov/data/gridded/data.narr.html.

What if we wanted to know the average height along a latitude or longitude? Perhaps we have the hypothesis that PBL height is higher at lower latitudes closer to the equator. How would we test this hypothesis? We could take the mean of the data along the axis of interest.

Since the PBL data is formatted in a 3D array, and we want to take the average along two of these dimensions to make a line plot of height versus dimension. If we want to know the average at each latitude, then we must take the mean along the longitude and time dimensions. If this is hard to visualize, think back to a rubix cube. We want to know the average at each point along the y-axis (the rows). In order to do this, we must smush the x-axis, so take the average of the columns. 

Once again, we can using different indexing methods in Xarray to accomplish the same task. Now let's plot the averages, this time using Xarray plotting.

In [ ]:
# find the mean of the data over time first
hpbl_timeavg = hpbl.mean(axis=0)
hpbl_timeavg.dims

In [ ]:
# find the mean of the rows (latitudes) using dimension names
hpbl_lat = hpbl_timeavg.mean('x')
hpbl_lat.dims

In [ ]:
fig = plt.figure(figsize=(10,6))
hpbl_lat.plot()
plt.show()

Conveniently, Xarray plotting can use the dimensions and the data variables to correctly label the axes. However, these labels aren't the best for professional presentations, so you can still use Matplotlib commands to clean up the figure. 

In [ ]:
fig = plt.figure(figsize=(10,6))

degrees = 1 + hpbl_lat['y']/111139 # convert x to degrees latitude
plt.plot(degrees,hpbl_lat)
plt.title('Average Planetary Boundary Layer Height')
# you can change the label data for your data

plt.xlabel('Latitude (˚)',fontsize=12)
plt.ylim((0,1000))
plt.ylabel('Height (m)',fontsize=12)
plt.show()

We hypothesized that PBL height was higher at lower latitude, and that appears to be true! At least, for the year 2020 when we average across all days and longitudes. 

### Merging data

So far, we've been manipulating data from one Dataset of data for 2020. What if we want to look at data from more than one year? Can we merge two or more Datasets together?

In [ ]:
# first, open another dataset
file2 = 'http://psl.noaa.gov/thredds/dodsC/Datasets/NARR/Dailies/monolevel/hpbl.2019.nc'

In [ ]:
# use xarray to open the file you chose
data2 = xr.open_dataset(file2,engine='netcdf4')

In [ ]:
# look at the data
data2

At first glance, this Dataset looks pretty similar to the first Dataset. The dimensions, coordinates, and data variables are the same. Let's combine this Dataset with the first Dataset, but create a new "date" dimension.

In [ ]:
# why are the Datasets listed data2, data1? What happens if you flip the order?
merge = xr.concat([data2,data],dim='time')
merge

We now have one larger Dataset that includes data and data2. These two original Datasets are differentiated by their position in the time dimension. 

### Exercises

**1.** Select a subset of data from the merge Dataset that is   
a) all times, the first 200 longitudes, and all latitudes

b) all times, all latitudes, longitudes 100 to 200

c) all times, the 3rd longitude, latitudes 20 to 50

**2.** Select the hpbl data from the merge Dataset. Find the following means of this DataArray.   
a) Mean of the latitudes

b) Mean of the longitudes

c) Mean of each year (two values)

**3.** Open a third Dataset for a new year. Merge this Dataset with the first two Datasets along the time dimension. There are a few ways to do this, but the easiest way is to create a new Dataset using the `concat` command and list all three date files in the brackets.

In [ ]:
# open a new Dataset


In [ ]:
# merge the three date Datasets together


**4.** Find the average PBL height over time of the three merged Datasets. Make a line plot of height over time. 

In [ ]:
# find the mean


In [ ]:
# plot the data


### Possible Solutions

In [ ]:
# 1a
subset = merge[dict(x=slice(None,200))]
subset.dims

In [ ]:
# 1b
subset = merge[dict(x=slice(100,200))]
subset.dims

In [ ]:
# 1c
subset = merge[dict(x=3,y=slice(20,50))]
subset

In [ ]:
# 2
var = merge['hpbl']
var

In [ ]:
# 2a
var.mean(('y','time'))

In [ ]:
# 2b
var.mean(('x','time'))

In [ ]:
# 2c
year1 = var.sel(time=slice(pd.to_datetime('2019-01-01'),pd.to_datetime('2019-12-31'))).mean()
print(year1)
year2 = var.sel(time=slice(pd.to_datetime('2020-01-01'),pd.to_datetime('2020-12-31'))).mean()
print(year2)

In [ ]:
# 3
file3 = 'http://psl.noaa.gov/thredds/dodsC/Datasets/NARR/Dailies/monolevel/hpbl.2018.nc'
data3 = xr.open_dataset(file3,engine='netcdf4')

In [ ]:
bigmerge = xr.concat([data3,merge],dim='time')
bigmerge.dims

In [ ]:
# 4
hpbl = bigmerge['hpbl'].mean(['x','y'])
dates = pd.to_datetime(hpbl['time'])

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.plot(dates,hpbl)
plt.ylim((0,1250))
plt.ylabel('Height (m)',fontsize=12)
plt.xlabel('Date',fontsize=12)
plt.title('Average PBL Height from 2018-2020',fontsize=18)
plt.show()